In [1]:
# Imports
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from scipy.stats import shapiro, kstest, probplot
from statsmodels.graphics.gofplots import qqplot
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

### Carga dos Dados 

In [ ]:
# Abrir o dataset e checar a estrutura do mesmo
df_pontuacao = pd.read_csv('./datasets/pontuacao_teste.csv')
df_pontuacao.info()

### EDA

In [ ]:
# Medidas estatísticas das variáveis
df_pontuacao.describe()

In [ ]:
# Plot de Dispersão
# X = horas_estudos (var independente)
# y = pontuacao_teste (var dependente)
sns.scatterplot(data=df_pontuacao, x='horas_estudo', y='pontuacao_teste')

In [ ]:
# Verificar se há outliers para X
# Plot BoxPlot
sns.boxplot(df_pontuacao, y='horas_estudo')

In [ ]:
# Verificar se há outliers para Y
# Plot BoxPlot
sns.boxplot(df_pontuacao, y='pontuacao_teste')

In [ ]:
# Verificar Correlação - Pearson
sns.heatmap(df_pontuacao.corr('pearson'), annot=True)

In [ ]:
# Verificar Correlação - Pearson
sns.heatmap(df_pontuacao.corr('spearman'), annot=True)

In [ ]:
# Histograma das variáveis - X
sns.displot(df_pontuacao, x='horas_estudo')

In [ ]:
# Histograma das variáveis - y
sns.displot(df_pontuacao, x='pontuacao_teste')